# Integrazione dati: esercizi

In questo tutorial mapperemo degli agritur su Umap, seguendo grossomodo [quest' altro tutorial](https://www.coderdojotrento.it/risorse/openstreetmap-e-agritur/) di CoderDojo Trento. La differenza sostanziale sarà che in questo caso invece dei Google spreadsheet useremo Python.

1. prendere dati agritur da dati.trentino.it
2. leggere file CSV in Python
3. cercare automaticamente coordinate geografiche usando le web api di MapQuest / OpenStreetMap Nominatim
    - parsing formato json
4. scrivere il nuovo file CSV con i campi latitudine e longitudine riempiti
5. importare il file CSV in Umap
6. Inserire la mappa in Jupyter

## 1 Prendiamoci i dati

Per partire da casi concreti, come già fatto in precedenza andiamo a cercarci dei dati dal catalogo opendata `dati.trentino.it`. In questo caso sceglieremo un file dal dataset [Agritur del Trentino](http://dati.trentino.it/dataset/agriturismi-del-trentino). 

**DOMANDA 1.1**: Quale è la licenza del dataset? Possiamo farci tutto quello che vogliamo ? 

Nel dataset troviamo la risorsa [Elenco Agritur](http://dati.trentino.it/dataset/agriturismi-del-trentino/resource/bb9f3185-602d-44c5-9f06-fc1f7ae25038) che al suo interno contiene un link ad un file CSV.

**ATTENZIONE**: Per questo esercizio, **NON** usate il CSV dal sito, ma scaricate invece [agritur16_10_2014.csv a questo link](agritur16_10_2014.csv) che come dal nome contiene un file del 2014. 

**DOMANDA 1.2**: notate differenze tra il file del 2014 e quello corrente sul sito ? 

**DOMANDA 1.3**: Se realizzaste un programma per leggere questi file degli agriturismi scaricandoli periodicamente dal sito,  a lungo andare potrebbero insorgere problemi ?

Vediamo qua una anterprima del file `agritur16_10_2014.csv` (NOTA: i campi a destra sono tagliati) : 

```
N_prog;Num_archivio;data_rilascio_prima_autorizzazione;Nome_Impresa_agricola;A ...
1;10;11/11/1986;DALLAGO LUCIANO;no;38060;Aldeno;Via S. D'Acquisto n. 4;29251/9 ...
2;26;16/12/1986;ARMAN CRISTINA;no;38010;Faedo;Loc. Pineta - Maso Nello;137876/ ...
3;37;22/12/1986;INAMA FRANCESCO;no;38010;Sanzeno;Via Casalini n. 74;130041/96; ...
4;49;15/01/1987;MONTIBELLER VALTER;no;38050;Roncegno;Via Prose n. 1;138559/96; ...
```

**DOMANDA 1.4**: Ci sono le intestazioni ? Qual'è il separatore ? 

**DOMANDA 1.5**: Quanti indirizzi ci sono nel file ? Se volessimo posizionare gli agritur su una mappa, quali indirizzi useremmo ? Ci sono le coordinate geografiche?

**DOMANDA 1.6**: Prova ad aprire il file in Excel o LibreOffice Calc, facendo attenzione a specificare i separatori corretti e l'encoding (guarda l'header 'Società' , in particolare il carattere finale !)




## 2. Leggiamo il CSV

**DA FARE 2.1**: Prova a caricare il file CSV in Python usando le istruzioni già viste [nella seconda lezione](http://softpython.readthedocs.io/it/latest/exercises/data-formats/data-formats-exercises.html#File-CSV)

Fai attenzione al delimitatore e specifica `encoding='utf-8'` come parametro nella `open` !

**SOLUZIONE**

In [1]:
import csv
with open('agritur16_10_2014.csv', encoding='utf-8', newline='') as f:
    reader = csv.reader(f, delimiter=';')
    for row in reader:
        print(row)

['N_prog', 'Num_archivio', 'data_rilascio_prima_autorizzazione', 'Nome_Impresa_agricola', 'Azienda_zootecnica', 'CAP', 'sede_impresa_agricola', 'indirizzo_impresa_agricola', 'CCIAA', 'pref', 'tel', 'società', 'malga', 'PEC', 'indirizzo_e_mail', 'Altitudine', 'Latitudine', 'Longitudine', 'Comune_Sede_Agriturismo', 'Indirizzo_agriturismo', 'Denominazione_Agriturismo', 'N_appartamenti', 'N_stanze_in_appartamento', 'Tot_posti_letto_in_appartamento', 'N_bagni_appartamento', 'N_Stanze', 'Tot_posti_letto_in_stanze', 'N_bagni_in_stanze', 'N_tot_stanze_e_stanze_in_appartamento', 'Tot_posti_letto', 'Alloggio_in_appartamenti', 'Alloggio_in_stanze', 'Prima_colazione', 'Ristorazione_solo_per_alloggiati', 'Servizio_Ristorante_per_passanti', 'N_posti_tavola_interni_a_struttura', 'N_posti_tavola_esterni_alla_struttura', 'N_tot_posti_tavola', 'N_tot_Bagni', 'Agricampeggio', 'N_piazzole', 'N_tot_posti_in_campeggio', 'Fattoria_didattica', 'Altre_attività_ricreative', 'Degustazione', 'Classificazione_punt

## 3. Geocoding con webapi

### 3.1 MapQuest / OpenStreetMap Nominatim

Possiamo leggere i valori dal CSV, ma purtroppo notiamo che mancano le coordinate geografiche. Per ottenerle, possiamo usare i servizi di MapQuest, che ci offre gratutitamente un cosiddetto servizio di _geocoding_ : Dati degli indirizzi, ci ritornerà le loro coordinate geografiche usando OpenStreetMap (detto OSM per gli amici) come riferimento.

Per capire cosa potremmo avere indietro, cerchiamo su OpenStreetMap un agritur, tipo il Montibeller di Roncegno:

* Sito di OpenStreetMap, stringa di ricerca `Montibeller, Roncegno`:

    http://www.openstreetmap.org/search?query=Montibeller%2C%20Roncegno#map=19/46.04691/11.41157

* Sito di OpenStreetMap, stringa di ricerca `Via Prose n. 1, Roncegno`:

    http://www.openstreetmap.org/search?query=via%20Prose%20n.%201%2C%20Roncegno#map=18/46.04698/11.41101

**DOMANDA 3.1.1**: Le due stringhe trovano risultati diversi. Come mai? Manca forse qualche dato ad OpenStreetMap?

### 3.2 Webapi JSON

Abbiamo visto una bella rappresentazione grafica del punto sulla mappa. Però ci piacerebbe ottenere quell'informazione in Python. Come fare? Oggigiorno, tantissimi portali offrono la possibilità di leggere e scrivere informazioni programmaticamente tramite cosiddette 'API REST'. OpenStreetMap offre un servizio simile che potremmo usare, ma teniamo presente che OpenStreetMap è un servizio gratuito gestito principalmente su base volontaristica con risorse limitate.

In alternativa, conviene rivolgersi a servizi offerti da aziende commerciali che possono sostenere un traffico più elevato. Di interessante c'è MapQuest, che oltre a mappe commerciali,  offre anche lo stesso identico servizio di OpenStreetMap (può farlo perchè le condizioni di licenza di OSM, molto libere, lo permettono). L'unico vincolo è che per usare il servizio bisogna prima registrare una cosiddetta 'Api key' da passare al servizio ogni volta che lo usiamo. Questo consente a MapQuest di monitorare eventuali abusi del servizio (per più info, vedere le [condizioni di licenza di MapQuest](https://developer.mapquest.com/documentation/open/). Quando ci connettiamo a indirizzi che iniziano con `open.mapquestapi`, vuol dire che stiamo usando mappe di OpenStreetMap.



Specifichiamo un paio di parametri importanti delle nostre chiamate web:

In [2]:
api_key = "Er38WkJVmeOl5AvFIAzM6lBBq4uEdgvG"  # Usate questa key SOLO per fare questi esercizi!
url_base = "http://open.mapquestapi.com/nominatim/v1/search"


Provate a fare copia e incolla nel vostro broswer dell'indirizzo seguente:

In [3]:
print(url_base)

http://open.mapquestapi.com/nominatim/v1/search


MapQuest dovrebbe rispondervi così:

```
The AppKey submitted with this request is invalid.
```

Si è offeso perchè non gli abbiamo passato una api key. 

**IMPORTANTE: Per oggi, la api key ve la diamo noi, ma se usate il servizio per i vostri progetti, [registratevene una ](https://developer.mapquest.com/plan_purchase/steps/business_edition/business_edition_free/register)!!!!**


Proviamo ad aggiungere la api key, mettendo il tutto nel browser dovremmo vedere dei bottoni (notate che il primo parametro è sempre preceduto da il punto di domanda `?`) :

In [4]:
url_with_key = url_base + "?key=" + api_key
print(url_with_key)

http://open.mapquestapi.com/nominatim/v1/search?key=Er38WkJVmeOl5AvFIAzM6lBBq4uEdgvG


Se siamo riusciti a vedere dei bottoni, vuol dire che abbiamo superato il primo ostacolo. Ma Python di bottoni non ci capisce un tubo! Dobbiamo fornire a pyhton un formato più 'digeribile':

In [5]:
url_json = url_base + "?key=" + api_key + "&format=json"

Se proviamo nel browser questa nuova url, vedremo che è sparito tutto. Un JSON vuoto non è per niente interessante, ma almeno sono spariti i bottoni:

In [6]:
print(url_json)

http://open.mapquestapi.com/nominatim/v1/search?key=Er38WkJVmeOl5AvFIAzM6lBBq4uEdgvG&format=json


Siamo finalmente pronti per eseguire la nostra prima query via web api ! Nella nostra query, scriveremo in linguaggio naturale `Montibeller,Roncegno`:

In [7]:

query = "Montibeller,Roncegno"

url_complete = url_json = url_base + "?key=" + api_key + "&format=json"  + "&q=" + query 

print(url_complete)

http://open.mapquestapi.com/nominatim/v1/search?key=Er38WkJVmeOl5AvFIAzM6lBBq4uEdgvG&format=json&q=Montibeller,Roncegno


Adesso dovremmo vedere un risultato tipo questo:
    
```
[{"place_id":"6560673","licence":"Data \u00a9 OpenStreetMap contributors, ODbL 1.0. http:\/\/www.openstreetmap.org\/copyright","osm_type":"node","osm_id":"673194565","boundingbox":["46.0469105","46.0469105","11.4115734","11.4115734"],"lat":"46.0469105","lon":"11.4115734","display_name":"Montibeller, Via Prose, Alps, Salembis, Roncegno Terme, Comunit\u00e0 Valsugana e Tesino, Provincia autonoma di Trento, Trentino-Alto Adige - S\u00fcdtirol, 38051, Italy","class":"tourism","type":"hotel","importance":0.211,"icon":"http:\/\/ip-10-98-183-183.mq-us-east-1.ec2.aolcloud.net:8000\/nominatim\/v1\/images\/mapicons\/accommodation_hotel2.p.20.png"}]
```

provate a copia e incollare il risultato in un editor che supporta i JSON, salvate il file come `.json` e dite all'editor di riformattare il documento. Se tutto va bene, dovrebbe venirvi fuori un bell'albero ordinato così: 

```json
[
    {
        "place_id": "6560673",
        "licence": "Data \u00a9 OpenStreetMap contributors, ODbL 1.0. http:\/\/www.openstreetmap.org\/copyright",
        "osm_type": "node",
        "osm_id": "673194565",
        "boundingbox": [
            "46.0469105",
            "46.0469105",
            "11.4115734",
            "11.4115734"
        ],
        "lat": "46.0469105",
        "lon": "11.4115734",
        "display_name": "Montibeller, Via Prose, Alps, Salembis, Roncegno Terme, Comunit\u00e0 Valsugana e Tesino, TN, Trentino-Alto Adige - S\u00fcdtirol, 38051, Italy",
        "class": "tourism",
        "type": "hotel",
        "importance": 0.211,
        "icon": "http:\/\/ip-10-98-183-183.mq-us-east-1.ec2.aolcloud.net:8000\/nominatim\/v1\/images\/mapicons\/accommodation_hotel2.p.20.png"
    }
]
```


Mmm.. non sembra tanto diversa da una combinazione di liste e dizionari Python... Forse possiamo riuscire ad estrarre quel `lat` e `lon` senza neanche troppa fatica ...

### 3.3. Requests in Python

Per chiamare le webapi da Python, installimo la libreria `requests`:

- Anaconda: `conda install requests`
- Linux/Mac : `python3 -m pip install --user requests`

In [8]:
import requests

In [9]:
r = requests.get(url_complete)


In [10]:
print(r)

<Response [200]>


In [11]:
r.text

'[{"place_id":"6560673","licence":"Data \\u00a9 OpenStreetMap contributors, ODbL 1.0. http:\\/\\/www.openstreetmap.org\\/copyright","osm_type":"node","osm_id":"673194565","boundingbox":["46.0469105","46.0469105","11.4115734","11.4115734"],"lat":"46.0469105","lon":"11.4115734","display_name":"Montibeller, Via Prose, Alps, Salembis, Roncegno Terme, Comunit\\u00e0 Valsugana e Tesino, TN, TAA, 38051, Italia","class":"tourism","type":"hotel","importance":0.211,"icon":"http:\\/\\/ip-10-98-171-30.mq-us-east-1.ec2.aolcloud.net:8000\\/nominatim\\/v1\\/images\\/mapicons\\/accommodation_hotel2.p.20.png"}]'

In [12]:
type(r.text)

str

In [13]:
r.headers['content-type']

'application/json; charset=UTF-8'

In [14]:
r.encoding

'UTF-8'

In [15]:
r.json()

[{'boundingbox': ['46.0469105', '46.0469105', '11.4115734', '11.4115734'],
  'class': 'tourism',
  'display_name': 'Montibeller, Via Prose, Alps, Salembis, Roncegno Terme, Comunità Valsugana e Tesino, TN, TAA, 38051, Italia',
  'icon': 'http://ip-10-98-171-30.mq-us-east-1.ec2.aolcloud.net:8000/nominatim/v1/images/mapicons/accommodation_hotel2.p.20.png',
  'importance': 0.211,
  'lat': '46.0469105',
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://www.openstreetmap.org/copyright',
  'lon': '11.4115734',
  'osm_id': '673194565',
  'osm_type': 'node',
  'place_id': '6560673',
  'type': 'hotel'}]

In [16]:
type(r.json())

list

In [17]:
r.json()[0]

{'boundingbox': ['46.0469105', '46.0469105', '11.4115734', '11.4115734'],
 'class': 'tourism',
 'display_name': 'Montibeller, Via Prose, Alps, Salembis, Roncegno Terme, Comunità Valsugana e Tesino, TN, TAA, 38051, Italia',
 'icon': 'http://ip-10-98-171-30.mq-us-east-1.ec2.aolcloud.net:8000/nominatim/v1/images/mapicons/accommodation_hotel2.p.20.png',
 'importance': 0.211,
 'lat': '46.0469105',
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://www.openstreetmap.org/copyright',
 'lon': '11.4115734',
 'osm_id': '673194565',
 'osm_type': 'node',
 'place_id': '6560673',
 'type': 'hotel'}

In [18]:
type(r.json()[0])

dict

In [19]:
r.json()[0]['lat']

'46.0469105'

In [20]:
r.json()[0]['lon']

'11.4115734'

**DA FARE 3.3.1**: prova a chiamare `requests.get` passandogli una URL sbagliata, come `boh` o parzialmente giusta come `http://open.mapquestapi.com/BLA`. Cosa ottieni di ritorno per i vari campi di `r`? 

### 3.4. Funzioni geocode per requests

In [21]:
def geocode_generic(address):
    # 'payload' è una variabile che ci definiamo noi, per metterci più comodamente i parametri 
    payload = {'key': api_key,  # Questa è la chiave lunga tipo Er38Wk... che abbiamo definito più sopra
               'format': 'json', 
               'q' : address}
    r = requests.get(url_base, params=payload)  # qua passiamo il 'payload' alla libreria requests
    print(r.url) # stampa l'url che requests ha usato 
    return r.json()


In [22]:
geocode_generic("Montibeller, Roncegno")

http://open.mapquestapi.com/nominatim/v1/search?format=json&q=Montibeller%2C+Roncegno&key=Er38WkJVmeOl5AvFIAzM6lBBq4uEdgvG


[{'boundingbox': ['46.0469105', '46.0469105', '11.4115734', '11.4115734'],
  'class': 'tourism',
  'display_name': 'Montibeller, Via Prose, Alps, Salembis, Roncegno Terme, Comunità Valsugana e Tesino, TN, TAA, 38051, Italia',
  'icon': 'http://ip-10-98-161-100.mq-us-east-1.ec2.aolcloud.net:8000/nominatim/v1/images/mapicons/accommodation_hotel2.p.20.png',
  'importance': 0.211,
  'lat': '46.0469105',
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://www.openstreetmap.org/copyright',
  'lon': '11.4115734',
  'osm_id': '673194565',
  'osm_type': 'node',
  'place_id': '6560673',
  'type': 'hotel'}]

In [23]:
json = geocode_generic("Montibeller, Via Prose n. 1, Roncegno")

http://open.mapquestapi.com/nominatim/v1/search?format=json&q=Montibeller%2C+Via+Prose+n.+1%2C+Roncegno&key=Er38WkJVmeOl5AvFIAzM6lBBq4uEdgvG


In [24]:
print(json)

[]


Per verificare se abbiamo trovato o meno qualcosa, possiamo controllare che la lunghezza della lista ritornata sia zero con `len`: 

In [25]:
if len(json) == 0:
    print("non ho trovato niente!")
else: 
    print("ho trovato !")

non ho trovato niente!


In [26]:
geocode_generic("Via Prose n. 1, Roncegno")

http://open.mapquestapi.com/nominatim/v1/search?format=json&q=Via+Prose+n.+1%2C+Roncegno&key=Er38WkJVmeOl5AvFIAzM6lBBq4uEdgvG


[{'boundingbox': ['46.0464048', '46.047558', '11.4100616', '11.4119591'],
  'class': 'highway',
  'display_name': 'Via Prose, Alps, Salembis, Roncegno Terme, Comunità Valsugana e Tesino, TN, TAA, 38051, Italia',
  'importance': 0.5,
  'lat': '46.0469731',
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://www.openstreetmap.org/copyright',
  'lon': '11.4109589',
  'osm_id': '149123476',
  'osm_type': 'way',
  'place_id': '102955753',
  'type': 'residential'}]

In [27]:
geocode_generic("Trento")

http://open.mapquestapi.com/nominatim/v1/search?format=json&q=Trento&key=Er38WkJVmeOl5AvFIAzM6lBBq4uEdgvG


[{'boundingbox': ['45.9775306', '46.1530112', '11.0224735', '11.1948226'],
  'class': 'place',
  'display_name': 'Trento, TN, TAA, Italia',
  'icon': 'http://ip-10-98-174-148.mq-us-east-1.ec2.aolcloud.net:8000/nominatim/v1/images/mapicons/poi_place_city.p.20.png',
  'importance': 0.72458366717332,
  'lat': '46.0664228',
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://www.openstreetmap.org/copyright',
  'lon': '11.1257601',
  'osm_id': '46663',
  'osm_type': 'relation',
  'place_id': '151138922',
  'type': 'city'},
 {'boundingbox': ['7.9378151', '8.292381', '126.00048', '126.355208'],
  'class': 'boundary',
  'display_name': 'Trento, Agusan Del Sur, Caraga, Philippine Continental Shelf',
  'icon': 'http://ip-10-98-174-148.mq-us-east-1.ec2.aolcloud.net:8000/nominatim/v1/images/mapicons/poi_boundary_administrative.p.20.png',
  'importance': 0.56,
  'lat': '8.114415',
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://www.openstreetmap.org/copyright',
  'lo

La [API di Nominatim](https://wiki.openstreetmap.org/wiki/Nominatim#Special_Keywords) (ricordiamo che Nominatim è il search engine di OpenStreetMap) ci permette di essere più specifici nei parametri che passiamo. Per esempio, si possono passare i parameteri `street` e `county`:  

In [28]:

def geocode_street_county(street, county):
    # 'payload' è una variabile che ci definiamo noi, per metterci più comodamente i parametri 
    payload = {'key': api_key,  # api_key è la chiave lunga tipo Er38Wk... che abbiamo definito più sopra
               'format': 'json',
               'street' : street,
               'county' : county}
    r = requests.get(url_base, params=payload)  # qua passiamo il 'payload' alla libreria requests
    print(r.url) # stampa l'url che requests ha usato 
    return r.json()



In [29]:
geocode_street_county("Via Prose n. 1", "Roncegno")

http://open.mapquestapi.com/nominatim/v1/search?format=json&county=Roncegno&key=Er38WkJVmeOl5AvFIAzM6lBBq4uEdgvG&street=Via+Prose+n.+1


[{'boundingbox': ['46.0464048', '46.047558', '11.4100616', '11.4119591'],
  'class': 'highway',
  'display_name': 'Via Prose, Alps, Salembis, Roncegno Terme, Comunità Valsugana e Tesino, TN, TAA, 38051, Italia',
  'importance': 0.5,
  'lat': '46.0469731',
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://www.openstreetmap.org/copyright',
  'lon': '11.4109589',
  'osm_id': '149123476',
  'osm_type': 'way',
  'place_id': '102955753',
  'type': 'residential'}]

## 4. Scriviamo un nuovo CSV con i campi lat e long

Adesso che abbiamo capito come prenderci le coordinate, possiamo

1. aprire in scrittura un nuovo CSV chiamato `agritur_mini_latlon.csv`
2. leggere il CSV originale `agritur_mini.csv`
3. per ogni riga del CSV originale `agritur_mini.csv`, leggere `lat` e `lon` da MapQuest
4. scrivere nel nuovo CSV `agritur_mini_latlon.csv` i campi vecchi più le nuove coordinate

<div class="alert alert-warning">

**Attenzione**: Per non sovraccaricare MapQuest, negli esercizi seguenti useremo **SOLO** il file [agritur_mini.csv](agritur_mini.csv) che contiene i primi 7 Agritur. Per un paio di questi sarà possibile trovare una georeferenziazione. Il CSV 'soluzione' lo potete vedere nel file [agritur_soluzione.csv](agritur_soluzione.csv) (NON modificatelo !)
</div>

Intanto scriviamo un programmino per leggere dall'input `agritur_mini.csv` e copiare tutto quanto letto in un file di output che chiameremo `agritur_mini_latlon.csv` 

In [30]:

import csv

# apriamo il file `agritur_mini_latlon`in scrittura (fouput è un nome  scelto da noi)
with open('agritur_mini_latlon.csv', 'w', encoding='utf-8') as foutput:   
    writer = csv.writer(foutput)  # Ci serve creare un'oggetto 'scrittore'
    
    # apriamo il file `agritur_mini`in scrittura (finput è un nome  scelto da noi)
    with open('agritur_mini.csv', encoding='utf-8', newline='') as finput:
        reader = csv.reader(finput, delimiter=';')  # delimitatore ';'
        for row in reader:
            writer.writerow(row)  # chiamiamo l'oggetto scrittore dicendogli di scrivere la riga appena letta

**DA FARE 4.1**: Copia a mano qua sotto il codice qua sopra, e usa `Control+Invio` per eseguirlo

In [31]:
# scrivi il codice

**DA FARE 4.2**: prova a cancellare il file `agritur_mini_latlon.csv`, eseguire la cella qua sopra e verificare che il programma effettivamente crei il file

**DOMANDA 4.3**: di default, il `writer` che separatori usa? Guarda il file di risultato.

**DA FARE 4.4**: Prova ad aggiungere un contatore per verificare a che riga siamo, poi tanto per capire dove sta la latitudine, prova a modificare `row` prima che venga scritta, in modo che il campo `Latitudine` (`row[16]`) e `Longitudine` (`row[17]`) siano messi rispettivamente a `123` e `456`

**SOLUZIONE**

In [32]:

import csv

# apriamo il file `agritur_mini_latlon`in scrittura (fouput è un nome  scelto da noi)
with open('agritur_mini_latlon.csv', 'w', encoding='utf-8') as foutput:   
    writer = csv.writer(foutput)  # Ci serve creare un'oggetto 'scrittore'
    
    # apriamo il file `agritur_mini`in lettura (finput è un nome  scelto da noi)
    with open('agritur_mini.csv', encoding='utf-8', newline='') as finput:
        reader = csv.reader(finput, delimiter=';')  # delimitatore ';'
        i = 0
        for row in reader:
            if i > 0:
                row[16] = 123
                row[17] = 456
            writer.writerow(row)  #chiamiamo l'oggetto scrittore dicendogli di scrivere la riga appena letta
            i += 1


**DA FARE 4.5**: Mentre leggi il CSV, adesso setta latitudine e longitudine usando risultati ottenuti chiamando geocode_street_county. Per gli input, puoi usare questi indici:

* `Indirizzo_agriturismo` : `row[19]`
* `Comune_Sede_Agriturismo` : `row[18]`

(ci sarebbe la `Denominazione_Agriturismo` : `row[20]` ma come visto prima funziona solo se l'agriturismo è già in OpenStreetMap, usando solo la via abbiamo qualche probabilità in più di successo)


**SOLUZIONE**

In [33]:

import csv

# apriamo il file `agritur_mini_latlon`in scrittura (fouput è un nome  scelto da noi)
with open('agritur_mini_latlon.csv', 'w', encoding='utf-8') as foutput:   
    writer = csv.writer(foutput)  # Ci serve creare un'oggetto 'scrittore'
    
    # apriamo il file `agritur_mini`in lettura (finput è un nome  scelto da noi)
    with open('agritur_mini.csv', encoding='utf-8', newline='') as finput:
        reader = csv.reader(finput, delimiter=';')  # delimitatore ';'
        i = 0
        for row in reader:
            if i > 0:
                json = geocode_street_county(row[19], row[18])
                if len(json) > 0:
                    print('trovato !')
                    row[16] = json[0]['lat']
                    row[17] = json[0]['lon']
            writer.writerow(row)  # chiamiamo l'oggetto scrittore dicendogli di scrivere la riga appena letta
            i += 1

http://open.mapquestapi.com/nominatim/v1/search?format=json&county=Garniga+Terme&key=Er38WkJVmeOl5AvFIAzM6lBBq4uEdgvG&street=Loc.+al+Lago+n.+15
http://open.mapquestapi.com/nominatim/v1/search?format=json&county=Faedo&key=Er38WkJVmeOl5AvFIAzM6lBBq4uEdgvG&street=Loc.+Pineta+-+Maso+Nello
http://open.mapquestapi.com/nominatim/v1/search?format=json&county=Sanzeno&key=Er38WkJVmeOl5AvFIAzM6lBBq4uEdgvG&street=Via+Casalini+n.+74
http://open.mapquestapi.com/nominatim/v1/search?format=json&county=Roncegno&key=Er38WkJVmeOl5AvFIAzM6lBBq4uEdgvG&street=Via+Prose+n.+1
trovato !
http://open.mapquestapi.com/nominatim/v1/search?format=json&county=Tenno&key=Er38WkJVmeOl5AvFIAzM6lBBq4uEdgvG&street=Loc.+Fontanelle
http://open.mapquestapi.com/nominatim/v1/search?format=json&county=Pergine+Valsugana&key=Er38WkJVmeOl5AvFIAzM6lBBq4uEdgvG&street=Via+Chimelli+n.+25
trovato !
http://open.mapquestapi.com/nominatim/v1/search?format=json&county=Coredo&key=Er38WkJVmeOl5AvFIAzM6lBBq4uEdgvG&street=Via+G.+Inama+n.+21


## 5. Importiamo in UMap

**DA FARE 5.1** Importa il CSV in Umap, seguendo le indicazioni di [quest'altro tutorial](https://docs.google.com/presentation/d/1CWo9pFl6jcR1EmDAXOmNeOayfyjfLqLR5-h5U8zxrrk/edit)

Il risultato finale dovrebbe essere così: [Mappa Agritur mini](http://umap.openstreetmap.fr/en/map/agritur-mini_181977#10/46.0594/11.3805)


In [37]:
%%HTML

<iframe width="100%" height="300px" frameBorder="0" src="http://umap.openstreetmap.fr/en/map/mia-mappa-agritur_182055?scaleControl=false&miniMap=false&scrollWheelZoom=false&zoomControl=true&allowEdit=false&moreControl=true&searchControl=null&tilelayersControl=null&embedControl=null&datalayersControl=true&onLoadPanel=undefined&captionBar=false#11/46.0966/11.4024"></iframe><p><a href="http://umap.openstreetmap.fr/en/map/mia-mappa-agritur_182055">See full screen</a></p>

**DA FARE 5.2**: Crea una nuova cella qua sotto, e prova ad incorporare nel foglio Jupyter la tua mappa come fatto qua sopra: